In [ ]:
!pip install openai


In [ ]:
import openai
openai.api_key = ''

In [ ]:
!pip install git+https://github.com/huggingface/transformers.git sentencepiece


  Cloning https://github.com/huggingface/transformers.git to /tmp/pip-req-build-n08wa3ab
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers.git /tmp/pip-req-build-n08wa3ab
  Resolved https://github.com/huggingface/transformers.git to commit f39627125b3778f98bbb202525790cf33a684c54
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [1]:
import torch

In [ ]:
from transformers import SeamlessM4Tv2Model, AutoProcessor


# Load the processor and model
processor = AutoProcessor.from_pretrained("facebook/seamless-m4t-v2-large")
model = SeamlessM4Tv2Model.from_pretrained("facebook/seamless-m4t-v2-large")

# Setup device
device = "cuda:0" if torch.cuda.is_available() else "cpu"
model.to(device)

/usr/local/lib/python3.10/dist-packages/transformers/deepspeed.py:23: FutureWarning: transformers.deepspeed module is deprecated and will be removed in a future version. Please import deepspeed modules directly from transformers.integrations
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tune

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

SeamlessM4Tv2Model(
  (shared): Embedding(256102, 1024, padding_idx=0)
  (text_encoder): SeamlessM4Tv2Encoder(
    (embed_tokens): Embedding(256102, 1024, padding_idx=0)
    (embed_positions): SeamlessM4Tv2SinusoidalPositionalEmbedding()
    (layers): ModuleList(
      (0-23): 24 x SeamlessM4Tv2EncoderLayer(
        (self_attn): SeamlessM4Tv2Attention(
          (k_proj): Linear(in_features=1024, out_features=1024, bias=True)
          (v_proj): Linear(in_features=1024, out_features=1024, bias=True)
          (q_proj): Linear(in_features=1024, out_features=1024, bias=True)
          (out_proj): Linear(in_features=1024, out_features=1024, bias=True)
        )
        (attn_dropout): Dropout(p=0.1, inplace=False)
        (self_attn_layer_norm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
        (ffn): SeamlessM4Tv2FeedForwardNetwork(
          (fc1): Linear(in_features=1024, out_features=8192, bias=True)
          (fc2): Linear(in_features=8192, out_features=1024, bias=True)


In [3]:
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 11.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 18.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 25.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 19.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 388.9/388.9 kB 40.0 MB/s eta 0:00:00
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.20.3
    Uninstalling huggingface-hub-0.20.3:
      Successfully uninstalled huggingface-hub-0.20.3


In [4]:
from datasets import load_dataset


In [5]:
dataset = load_dataset("Ehtisham1328/urdu-idioms-with-english-translation", split='train')


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/2111 [00:00<?, ? examples/s]

In [ ]:
print(dataset.features);

{'محنت کامیابی کی کنجی ہے۔': Value(dtype='string', id=None), 'Hard work is the key to success.': Value(dtype='string', id=None)}


In [6]:
urdu_key = 'محنت کامیابی کی کنجی ہے۔'
english_key = 'Hard work is the key to success.'

In [7]:
idioms = [(entry[urdu_key], entry[english_key]) for entry in dataset]

In [ ]:
translated_idioms = []
count = 0
for urdu_idiom, expected_english in idioms:
  if count == 500:
    break
  count+=1
  print(count)
  # Process the English idiom
  text_inputs = processor(text=urdu_idiom, src_lang="urd", return_tensors="pt").to(device)

  # Generate translation to english without generating speech
  output_tokens = model.generate(**text_inputs, tgt_lang="eng", generate_speech=False)
  #print(output_tokens[0].tolist())
  translated_text = processor.decode(output_tokens[0].tolist()[0], skip_special_tokens=True)

  translated_idioms.append((urdu_idiom, translated_text, expected_english))

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


In [8]:
!pip install huggingface_hub

from huggingface_hub import notebook_login

notebook_login()

In [9]:
!pip install accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.6/297.6 kB 6.9 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.19.3-py3-none-manylinux1_x86_64.whl (166.0 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-many

In [10]:
from transformers import pipeline

model_id = "meta-llama/Meta-Llama-3-8B-Instruct"
translate_pipeline = pipeline(
  "text-generation",
  model=model_id,
  model_kwargs={"torch_dtype": torch.bfloat16},
  device_map="auto",
)

config.json:   0%|          | 0.00/654 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/59.6k [00:00<?, ?B/s]

model-00001-of-00030.safetensors:   0%|          | 0.00/4.58G [00:00<?, ?B/s]

model-00002-of-00030.safetensors:   0%|          | 0.00/4.66G [00:00<?, ?B/s]

model-00003-of-00030.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

...

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1003: UserWarning: Not enough free disk space to download the file. The expected file size is: 4664.17 MB. The target location /root/.cache/huggingface/hub only has 523.06 MB free disk space.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1003: UserWarning: Not enough free disk space to download the file. The expected file size is: 4664.17 MB. The target location /root/.cache/huggingface/hub/models--meta-llama--Meta-Llama-3-70B-Instruct/blobs only has 523.06 MB free disk space.
  warnings.warn(


model-00012-of-00030.safetensors:   0%|          | 0.00/4.66G [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1003: UserWarning: Not enough free disk space to download the file. The expected file size is: 4664.17 MB. The target location /root/.cache/huggingface/hub only has 0.00 MB free disk space.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1003: UserWarning: Not enough free disk space to download the file. The expected file size is: 4664.17 MB. The target location /root/.cache/huggingface/hub/models--meta-llama--Meta-Llama-3-70B-Instruct/blobs only has 0.00 MB free disk space.
  warnings.warn(


model-00012-of-00030.safetensors:   0%|          | 0.00/4.66G [00:00<?, ?B/s]

ValueError: Could not load model meta-llama/Meta-Llama-3-70B-Instruct with any of the following classes: (<class 'transformers.models.auto.modeling_auto.AutoModelForCausalLM'>, <class 'transformers.models.auto.modeling_tf_auto.TFAutoModelForCausalLM'>, <class 'transformers.models.llama.modeling_llama.LlamaForCausalLM'>). See the original errors:

while loading with AutoModelForCausalLM, an error is thrown:
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py", line 283, in infer_framework_load_model
    model = model_class.from_pretrained(model, **kwargs)
  File "/usr/local/lib/python3.10/dist-packages/transformers/models/auto/auto_factory.py", line 563, in from_pretrained
    return model_class.from_pretrained(
  File "/usr/local/lib/python3.10/dist-packages/transformers/modeling_utils.py", line 3436, in from_pretrained
    resolved_archive_file, sharded_metadata = get_checkpoint_shard_files(
  File "/usr/local/lib/python3.1

In [39]:
def translate(text, pipeline):
  messages = [
      {"role": "system", "content": "You are a translator that is aware of idioms, colloquialisms, and figurative language. You take the input and Only output the English Translation. NOTHING ELSE!"},
      {"role": "user", "content": text}
  ]
  prompt = pipeline.tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True
  )

  # Define end-of-sentence terminators
  terminators = [
    pipeline.tokenizer.eos_token_id,
    pipeline.tokenizer.convert_tokens_to_ids("<|eot_id|>")  # Adjust or remove as necessary
  ]

  # Generate output with specific control on sampling
  results = pipeline(
    prompt,
    max_new_tokens=256,
    eos_token_id=terminators,
    do_sample=True,
    temperature=0.6,
    top_p=0.9
  )
  # prompt = f"Translate this to English: {text}"
  # results = pipeline(messages, max_length=60, num_return_sequences=1)
  translated_text = results[0]["generated_text"][len(prompt):]
  print(translated_text)
  return translated_text

In [52]:
translated_idioms_llama = []
count = 0

for urdu_idiom, expected_english in idioms:
  if count == 500:
    break
  count+=1
  print(count)
  translated_text = translate(urdu_idiom, translate_pipeline)
  translated_idioms_llama.append((urdu_idiom, translated_text, expected_english))

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


1


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Men's courage is God's help.
2


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


...

Where has my courage gone?
498


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


To and fro.
499


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


To cut a long story short.
500
In sheep's clothing


In [42]:
from nltk.translate.bleu_score import sentence_bleu

In [43]:
!pip install nltk
import nltk
from nltk.translate.bleu_score import corpus_bleu
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [44]:
def calculate_bleu(translated_idioms):
  references = []
  candidates = []
  for _, translated_text, expected_english in translated_idioms:
    # Tokenize the expected and generated translations
    reference = nltk.word_tokenize(expected_english.lower())
    candidate = nltk.word_tokenize(translated_text.lower())

    # For corpus BLEU scores, references are lists of lists, candidates are lists
    references.append([reference])  # reference needs to be a list of lists
    candidates.append(candidate)

  # Calculate the BLEU score
  bleu_score = corpus_bleu(references, candidates)
  return bleu_score

def calculate_exact_match_score(translated_idioms):
  exact_matches = sum(1 for _, translated, expected in translated_idioms if translated.strip().lower() == expected.strip().lower())
  return exact_matches / len(translated_idioms)

def weighted_score(bleu_score, exact_match_score, bleu_weight=0.5, exact_match_weight=0.5):
  return (bleu_score * bleu_weight) + (exact_match_score * exact_match_weight)

In [53]:
bleu_score = calculate_bleu(translated_idioms)
exact_match_score = calculate_exact_match_score(translated_idioms)
final_score = weighted_score(bleu_score, exact_match_score, bleu_weight=1, exact_match_weight=0)
print(f"Combined Score: {final_score:.2f}")

NameError: name 'translated_idioms' is not defined

In [ ]:
print(translated_idioms)

[('ہمتِ مرداں مددِ خدا۔', 'We are dead, God help us.', 'The courage of men is aided by God.'), ('کام بِن دام نہیں۔', "It's not a job.", 'Nothing comes without a price.'), ('مشق انسان کو کامل بنا دیتی ہے۔', 'Practice makes perfect.', 'Practice makes a person perfect.'), ('جہاں چاہ وہاں راہ۔', 'Wherever there is a way.', 'Where there is a will, there is a way.'), ('انت بھلا تو سب بھلا۔', 'All is well.', 'If the end is good, everything is good.'), ('زندگی پھولوں کی سیج نہیں۔', 'Life is not a bouquet of flowers.', 'Life is not a bed of roses.'), ('زندگی زندہ دلی کا نام ہے۔', 'Life is the name of the living heart.', 'Life is the name of a lively heart.'), ('تجربہ سب سے بڑا استاد ہے۔', 'Experience is the greatest teacher.', 'Experience is the greatest teacher.'), ('دودھ کا جلا چھاچھ بھی پھونک پھونک کر پیتا ہے۔', 'Milk is also blown away by the wind.', 'Even a burnt milk pudding is consumed bit by bit.'), ('وقت کسی کا انتظار نہیں کرتا۔', 'Time does not wait for anyone.', 'Time waits for no on

In [54]:
print(translated_idioms_llama)

[('ہمتِ مرداں مددِ خدا۔', "Men's courage is God's help.", 'The courage of men is aided by God.'), ('کام بِن دام نہیں۔', "It's all or nothing.", 'Nothing comes without a price.'), ('مشق انسان کو کامل بنا دیتی ہے۔', 'Human life is built around hard work.', 'Practice makes a person perfect.'), ('جہاں چاہ وہاں راہ۔', "Where there's a will, there's a way.", 'Where there is a will, there is a way.'), ('انت بھلا تو سب بھلا۔', "You're as good as everyone else.", 'If the end is good, everything is good.'), ('زندگی پھولوں کی سیج نہیں۔', 'Life is not a bed of roses.', 'Life is not a bed of roses.'), ('زندگی زندہ دلی کا نام ہے۔', 'Life is the name of the heart.', 'Life is the name of a lively heart.'), ('تجربہ سب سے بڑا استاد ہے۔', 'Experience is the greatest teacher.', 'Experience is the greatest teacher.'), ('دودھ کا جلا چھاچھ بھی پھونک پھونک کر پیتا ہے۔', 'The milk is being gulped down with a loud slurping sound.', 'Even a burnt milk pudding is consumed bit by bit.'), ('وقت کسی کا انتظار نہیں ک

In [55]:
bleu_score_llama = calculate_bleu(translated_idioms_llama)
exact_match_score_llama = calculate_exact_match_score(translated_idioms_llama)
final_score_llama = weighted_score(bleu_score_llama, exact_match_score_llama, bleu_weight=1, exact_match_weight=0)
print(f"Combined Score: {final_score_llama:.2f}")

Combined Score: 0.06
